In [8]:
import pandas as pd 
import numpy as np
import math
import tensorflow as tf
print(pd.__version__)
import random
import matplotlib.pyplot as plt

1.2.0


In [9]:
from keras.models import Sequential
from keras.layers.core import Dense
import keras.backend as K
from keras import optimizers
from keras import models
from keras import layers
from tensorflow import keras 

In [10]:
model_regr = keras.models.load_model('regr.h5')
model_class = keras.models.load_model('class.h5')

In [11]:
def gen_electron(dimx, dimy, p1):
    x1 = random.uniform(dimx[0], dimx[1])
    x2 = random.uniform(dimy[0], dimy[1])
    return np.array([p1, 0, 0, x1, x2])

In [12]:
def gen_photon(dimx, dimy, modulus):
    x1 = random.uniform(dimx[0], dimx[1])
    x2 = random.uniform(dimy[0], dimy[1])
    costheta = random.uniform(-1, 1)
    cosphi = random.random()
    proj_xy = cosphi*modulus
    return np.array([proj_xy*costheta, proj_xy*np.sqrt(1-costheta**2), modulus*np.sqrt(1-cosphi**2), x1, x2])

In [16]:
x = 24
y = 24
p_e = 0.1
p_pho = 0.1  
nodes_x = 40
nodes_y = 40
dx = x/nodes_x
dy = y/nodes_y
per_node = 100
threshold = 0.5

In [ ]:
photon_final = []
for node_x in range(nodes_x):
    print("node_x n: {0} out of {1}".format(node_x, nodes_x))
    lim_x_min = dx* node_x
    lim_x_max = (dx+1)*node_x
    for node_y in range(nodes_y):
        print("node_y n: {0} out of {1}".format(node_y, nodes_y))
        for n in range(per_node):
            lim_y_min = dy* node_y
            lim_y_max = (dy+1)*node_y
            electron = gen_electron([lim_x_min, lim_x_max], [lim_y_min, lim_y_max], p_e)
            photon = gen_photon([lim_x_min, lim_x_max], [lim_y_min, lim_y_max], p_pho)
            class_x = np.array([[electron[0], electron[1], electron[2],
                          photon[0], photon[1], photon[2],
                          electron[3], electron[4],
                          photon[3], photon[4]]])
            classification = model_class.predict(np.stack(class_x))
            if classification >= threshold:
                regr_x = np.array([class_x[0,:5]])
                regr_data = model_regr.predict(regr_x)
                photon_final.append(np.linalg.norm(regr_data[0]))
            else:
                photon_final.append(np.linalg.norm([photon[0], photon[1], photon[2]]))

node_x n: 0 out of 40
node_y n: 0 out of 40
node_y n: 1 out of 40
node_y n: 2 out of 40
node_y n: 3 out of 40
node_y n: 4 out of 40
node_y n: 5 out of 40
node_y n: 6 out of 40
node_y n: 7 out of 40
node_y n: 8 out of 40
node_y n: 9 out of 40
node_y n: 10 out of 40
node_y n: 11 out of 40
node_y n: 12 out of 40
node_y n: 13 out of 40
node_y n: 14 out of 40
node_y n: 15 out of 40
node_y n: 16 out of 40
node_y n: 17 out of 40
node_y n: 18 out of 40
node_y n: 19 out of 40
node_y n: 20 out of 40
node_y n: 21 out of 40
node_y n: 22 out of 40
node_y n: 23 out of 40
node_y n: 24 out of 40
node_y n: 25 out of 40
node_y n: 26 out of 40
node_y n: 27 out of 40
node_y n: 28 out of 40
node_y n: 29 out of 40
node_y n: 30 out of 40
node_y n: 31 out of 40
node_y n: 32 out of 40
node_y n: 33 out of 40
node_y n: 34 out of 40
node_y n: 35 out of 40
node_y n: 36 out of 40
node_y n: 37 out of 40
node_y n: 38 out of 40
node_y n: 39 out of 40
node_x n: 1 out of 40
node_y n: 0 out of 40
node_y n: 1 out of 40
no

node_y n: 32 out of 40
node_y n: 33 out of 40
node_y n: 34 out of 40
node_y n: 35 out of 40
node_y n: 36 out of 40
node_y n: 37 out of 40
node_y n: 38 out of 40
node_y n: 39 out of 40
node_x n: 9 out of 40
node_y n: 0 out of 40
node_y n: 1 out of 40
node_y n: 2 out of 40
node_y n: 3 out of 40
node_y n: 4 out of 40
node_y n: 5 out of 40
node_y n: 6 out of 40
node_y n: 7 out of 40
node_y n: 8 out of 40
node_y n: 9 out of 40
node_y n: 10 out of 40
node_y n: 11 out of 40
node_y n: 12 out of 40
node_y n: 13 out of 40
node_y n: 14 out of 40
node_y n: 15 out of 40
node_y n: 16 out of 40
node_y n: 17 out of 40
node_y n: 18 out of 40
node_y n: 19 out of 40
node_y n: 20 out of 40
node_y n: 21 out of 40
node_y n: 22 out of 40
node_y n: 23 out of 40
node_y n: 24 out of 40
node_y n: 25 out of 40
node_y n: 26 out of 40
node_y n: 27 out of 40
node_y n: 28 out of 40
node_y n: 29 out of 40
node_y n: 30 out of 40
node_y n: 31 out of 40
node_y n: 32 out of 40
node_y n: 33 out of 40
node_y n: 34 out of 40

node_y n: 24 out of 40
node_y n: 25 out of 40
node_y n: 26 out of 40
node_y n: 27 out of 40
node_y n: 28 out of 40
node_y n: 29 out of 40
node_y n: 30 out of 40
node_y n: 31 out of 40
node_y n: 32 out of 40
node_y n: 33 out of 40
node_y n: 34 out of 40
node_y n: 35 out of 40
node_y n: 36 out of 40
node_y n: 37 out of 40
node_y n: 38 out of 40
node_y n: 39 out of 40
node_x n: 18 out of 40
node_y n: 0 out of 40
node_y n: 1 out of 40
node_y n: 2 out of 40
node_y n: 3 out of 40
node_y n: 4 out of 40
node_y n: 5 out of 40
node_y n: 6 out of 40
node_y n: 7 out of 40
node_y n: 8 out of 40
node_y n: 9 out of 40
node_y n: 10 out of 40
node_y n: 11 out of 40
node_y n: 12 out of 40
node_y n: 13 out of 40
node_y n: 14 out of 40
node_y n: 15 out of 40
node_y n: 16 out of 40
node_y n: 17 out of 40
node_y n: 18 out of 40
node_y n: 19 out of 40
node_y n: 20 out of 40
node_y n: 21 out of 40
node_y n: 22 out of 40
node_y n: 23 out of 40
node_y n: 24 out of 40
node_y n: 25 out of 40
node_y n: 26 out of 4

node_y n: 16 out of 40
node_y n: 17 out of 40
node_y n: 18 out of 40
node_y n: 19 out of 40
node_y n: 20 out of 40
node_y n: 21 out of 40
node_y n: 22 out of 40
node_y n: 23 out of 40
node_y n: 24 out of 40
node_y n: 25 out of 40
node_y n: 26 out of 40
node_y n: 27 out of 40
node_y n: 28 out of 40
node_y n: 29 out of 40
node_y n: 30 out of 40
node_y n: 31 out of 40
node_y n: 32 out of 40
node_y n: 33 out of 40
node_y n: 34 out of 40
node_y n: 35 out of 40
node_y n: 36 out of 40
node_y n: 37 out of 40
node_y n: 38 out of 40
node_y n: 39 out of 40
node_x n: 27 out of 40
node_y n: 0 out of 40
node_y n: 1 out of 40
node_y n: 2 out of 40
node_y n: 3 out of 40
node_y n: 4 out of 40
node_y n: 5 out of 40
node_y n: 6 out of 40
node_y n: 7 out of 40
node_y n: 8 out of 40
node_y n: 9 out of 40
node_y n: 10 out of 40
node_y n: 11 out of 40
node_y n: 12 out of 40
node_y n: 13 out of 40
node_y n: 14 out of 40
node_y n: 15 out of 40
node_y n: 16 out of 40
node_y n: 17 out of 40
node_y n: 18 out of 4

In [ ]:
np.array(photon_final)

In [ ]:
def energy_spectrum(energy_array, bins):
    energy_array = np.array(energy_array)
    plt.hist(energy_array, bins)
    plt.yscale("log")
    plt.show()

In [ ]:
energy_spectrum(photon_final, 200)